In [4]:
import h5py
import glob

import numpy as np
from PIL import Image

trajectory_base_path = '/workspace/xarm-dataset/place_the_blue_box_on_plate'
trajectory_action_paths = glob.glob(f'{trajectory_base_path}*/action/*.hdf5')

all_trajectory_actions = []

for file_path in trajectory_action_paths:

    ### Load the trajectory with h5py
    with h5py.File(file_path, 'r') as f:
        ### Parse the image paths and load the corresponding images
        image_paths = [
            path.decode('utf-8').replace('/workspace/xarm-dataset/place_the_blue_box_on_plate', trajectory_base_path)
            for path in f['image_path'][:]
        ]

        images = []
        for img_path in image_paths:
            img = Image.open(img_path)
            img = img.resize((256, 256), resample=0)
            img_array = np.array(img)
            if len(img_array.shape) == 2:
                img_array = np.stack([img_array] * 3, axis=-1)
            images.append(img_array)

        ### Parse the action vectors from the trajectory
        actions = np.concatenate([
            f['abs_pos'][:], 
            f['abs_rot'][:], 
            f['gripper'][:][:, np.newaxis],
            
        ], axis=1)

        all_trajectory_actions.append(actions)